In [1]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import ipywidgets as widgets
from IPython.display import display, clear_output

# ------------------------------------------------------------------
#          CONFIGURACIÓN DE BIGQUERY CON TU CUENTA
# ------------------------------------------------------------------
SERVICE_ACCOUNT_FILE = "mimic-sergi.json"
PROJECT_ID = "ogi-uci-i61"

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE
)
client = bigquery.Client(credentials=credentials, project=PROJECT_ID)

# ------------------------------------------------------------------
#          DEFINICIÓN DE DATASETS Y TABLAS
# ------------------------------------------------------------------
hosp_dataset_id        = 'mimiciv_hosp'
icu_dataset_id         = 'mimiciv_icu'
d_items_table          = 'd_items'
d_labitems_table       = 'd_labitems'
chartevents_table      = 'chartevents'
procedureevents_table  = 'procedureevents'
labevents_table        = 'labevents'

# ------------------------------------------------------------------
#          WIDGETS DE INTERFAZ
# ------------------------------------------------------------------
search_mode = widgets.RadioButtons(
    options=[('Descripción', 'desc'), ('Código', 'code')],
    description='Buscar por:'
)
term_input = widgets.Text(
    description='Término:',
    placeholder='p.ej. White o 51755,51300'
)
search_btn = widgets.Button(description='Buscar')
out = widgets.Output()

# ------------------------------------------------------------------
#          FUNCIÓN PRINCIPAL DE BÚSQUEDA
# ------------------------------------------------------------------
def on_search_clicked(_):
    with out:
        clear_output()
        mode = search_mode.value
        term = term_input.value.strip()

        if not term:
            print("🔍 Introduce una descripción o códigos separados por comas.")
            return

        if mode == 'desc':
            kws = [kw.strip() for kw in term.split(',')]
            like_clauses = " OR ".join(
                f"LOWER(label) LIKE '%{kw.lower()}%'" for kw in kws
            )

            q_icu = f"""
                SELECT itemid, label
                FROM `{PROJECT_ID}.{icu_dataset_id}.{d_items_table}`
                WHERE {like_clauses}
            """
            df_icu = client.query(q_icu).to_dataframe()
            df_icu['base_de_datos'] = 'ICU'

            q_hosp = f"""
                SELECT itemid, label
                FROM `{PROJECT_ID}.{hosp_dataset_id}.{d_labitems_table}`
                WHERE {like_clauses}
            """
            df_hosp = client.query(q_hosp).to_dataframe()
            df_hosp['base_de_datos'] = 'HOSP'

            labels_df = pd.concat([df_icu, df_hosp], ignore_index=True)
            item_ids = tuple(labels_df['itemid'].unique())

        else:
            try:
                item_ids = tuple(int(x.strip()) for x in term.split(','))
            except ValueError:
                print("❌ Error: los códigos deben ser números separados por comas.")
                return

            if not item_ids:
                print("⚠️ No se encontraron ítems con ese criterio.")
                return

            item_ids_str = "(" + ", ".join(str(x) for x in item_ids) + ")"

            q1 = f"""
                SELECT itemid, label
                FROM `{PROJECT_ID}.{icu_dataset_id}.{d_items_table}`
                WHERE itemid IN {item_ids_str}
            """
            df1 = client.query(q1).to_dataframe()
            df1['base_de_datos'] = 'ICU'

            q2 = f"""
                SELECT itemid, label
                FROM `{PROJECT_ID}.{hosp_dataset_id}.{d_labitems_table}`
                WHERE itemid IN {item_ids_str}
            """
            df2 = client.query(q2).to_dataframe()
            df2['base_de_datos'] = 'HOSP'

            labels_df = pd.concat([df1, df2], ignore_index=True)

        if not item_ids:
            print("⚠️ No se encontraron ítems con ese criterio.")
            return

        item_ids_str = "(" + ", ".join(str(x) for x in item_ids) + ")"

        counts = []
        for table, alias, ds in [
            (chartevents_table,    'chartevents',    icu_dataset_id),
            (procedureevents_table,'procedureevents',icu_dataset_id),
            (labevents_table,      'labevents',      hosp_dataset_id),
        ]:
            q_cnt = f"""
                SELECT itemid, COUNT(*) AS num_records
                FROM `{PROJECT_ID}.{ds}.{table}`
                WHERE itemid IN {item_ids_str}
                GROUP BY itemid
            """
            df_cnt = client.query(q_cnt).to_dataframe()
            df_cnt['tabla_eventos'] = alias
            df_cnt = df_cnt.rename(columns={'num_records': 'número_registros'})
            counts.append(df_cnt)

        counts_df = pd.concat(counts, ignore_index=True)

        result_df = (
            labels_df
            .merge(counts_df, on='itemid', how='left')
            .fillna({'número_registros': 0})
            .rename(columns={
                'itemid': 'código',
                'label': 'etiqueta'
            })
        )[
            ['código', 'etiqueta', 'base_de_datos', 'tabla_eventos', 'número_registros']
        ]

        display(result_df)

# ------------------------------------------------------------------
#          INICIAR INTERFAZ
# ------------------------------------------------------------------
search_btn.on_click(on_search_clicked)
display(widgets.VBox([search_mode, term_input, search_btn, out]))
